## Funciones

### Guardar como geojson

In [11]:
import os
# pip install geopandas
import geopandas as gpd


def save_geojson(gdf, filename = 'test.geojson'):
    if not os.path.exists('./geojson/'):
        os.makedirs('./geojson/')
    
    try:    # Delete if geojson exists, cause overwriting is not supported
        os.remove('./geojson/'+filename)
    except OSError:
        pass
    
    gdf.to_file('./geojson/'+filename, driver="GeoJSON", encoding='utf-8')
    


### Subir archivo a Mapbox

In [ ]:
# pip install mapbox
from mapbox import Uploader
import json

def upload_file(data, name, username = 'your_mapbox_user', token = 'your_user_token'):
    try:    # Clear upload file if exists
        os.remove('./upload_data.geojson')
    except OSError:
        pass
    
    # Dump into file for upload    
    with open('./upload_data.geojson', 'w') as outfile:
        json.dump(data, outfile)

    service = Uploader(access_token=token)
    with open('./upload_data.geojson', 'rb') as src:
        # Acquisition of credentials, staging of data, and upload
        # finalization is done by a single method in the Python SDK.
        upload_resp = service.upload(src, username+'.'+name)
        

# Ejemplo de uso: 

#### Cargamos un archivo con las delimitaciones de Departamentos Argentinos que provee el Instituto Geografico:

In [12]:
from urllib.request import urlretrieve
# pip install pyunpack
from pyunpack import Archive

# Obtaining the data from the URL's
url_front = 'https://www.ign.gob.ar/descargas/geodatos/SHAPES/'

## Carpeta donde quedaran los shp
extract_dir = 'IGN_shp'
file_name = 'ign_provincia.zip'

full_file = os.path.join(os.getcwd(), extract_dir, file_name)

# Descarga el rar solo si no existe.
if not os.path.isfile(full_file):
    # Si no esta el directorio lo crea
    if not os.path.exists(extract_dir):
        os.makedirs(extract_dir)

    # Toma data del URL
    urlretrieve(url_front + file_name, full_file)

Archive(full_file).extractall(extract_dir)

In [13]:
# Aca pueden cargar otro archivo a eleccion.
provincias = gpd.read_file('./IGN_shp/Provincia/')
provincias['PROV_'] = provincias['IN1']

# Si queremos corregir el crs. Tiene que estar en lat lng para que lo acepte Mapbox.
# gdf.crs = ...; gdf = gdf.to_crs({'init': "EPSG:4326"})

## Subimos el geojson a nuestra cuenta de MapBox

In [14]:
name = 'test'
gdf = provincias

# En caso que falte alguna geometry
gdf = gdf.dropna(subset = ['geometry'])

save_geojson(gdf, name + '.geojson')

username = 'your_mapbox_user'
token = 'your_mapbox_token'

files = os.listdir('./geojson/')
files = [f for f in files if name in f]

names = [name.split('.')[0] for name in files]

# El siguiente loop subre todo lo que este en la carpeta geojson
for i in range(len(files)):
    print(names[i])
    data_ = json.load(open('./geojson/'+files[i]))

    try:
        upload_file(data_, names[i])
    except:
        pass

test
